<a href="https://colab.research.google.com/github/coldsober-irene/ASSIGNMENTS/blob/main/AHP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
import numpy as np
import pandas as pd
from copy import deepcopy as cp

In [128]:
from pandas.core.indexes.numeric import Int64Index
class AHP:
  def __init__(self, choices, criteria):
    self.choices = choices
    self.criteria = criteria
    self.pairwise_matrix = np.ones(shape = (len(self.criteria), len(self.criteria)))
    self.index_criteria = {index:criteria for index, criteria in enumerate(self.criteria)}
    self.representation = [str(k) + '. ' + v for k, v in self.index_criteria.items()]
    self.ratings = [i for i in range(1, 10)]

  def Pairwise1(self):
    pairs = []
    pair_matrix = {}
    for index, criteria in enumerate(self.criteria):
      print('comparison')
      print('_'*60)

      for i in range(index+1, len(self.criteria)):
        print()
        print(f'criteria'.upper())
        print('\n'.join(self.representation))
        print()
        choice = int(input(f'{criteria} - {self.criteria[i]} :'))
        rating = int(input('rating out of 9 : '))
        # UPDATE pairwise_matrix
        arrangement = []
        if i != index:
          if choice != index:
            self.pairwise_matrix[choice][index] = rating
            self.pairwise_matrix[index][choice] = 1 / rating
          else:
            self.pairwise_matrix[choice][i] = rating
            self.pairwise_matrix[i][choice] = 1 / rating
            print(f"CURRENT MATRIX: {self.pairwise_matrix}")
          print()
          pairs.append(criteria + "-" + self.criteria[i])

    return (self.pairwise_matrix, pd.DataFrame(self.pairwise_matrix, columns = self.criteria, index = self.criteria))

In [129]:
analytica = AHP(choices = ['Accord', 'Saturn', 'Cavalier'],
                criteria = ['price', 'mpg', 'comfort', 'style'])
# print(analytica.pairwise_matrix)
_,original_df = analytica.Pairwise1()
# print(analytica.pairwise_matrix)

comparison
____________________________________________________________

CRITERIA
0. price
1. mpg
2. comfort
3. style

price - mpg :0
rating out of 9 : 3
CURRENT MATRIX: [[1.         3.         1.         1.        ]
 [0.33333333 1.         1.         1.        ]
 [1.         1.         1.         1.        ]
 [1.         1.         1.         1.        ]]


CRITERIA
0. price
1. mpg
2. comfort
3. style

price - comfort :0
rating out of 9 : 2
CURRENT MATRIX: [[1.         3.         2.         1.        ]
 [0.33333333 1.         1.         1.        ]
 [0.5        1.         1.         1.        ]
 [1.         1.         1.         1.        ]]


CRITERIA
0. price
1. mpg
2. comfort
3. style

price - style :0
rating out of 9 : 2
CURRENT MATRIX: [[1.         3.         2.         2.        ]
 [0.33333333 1.         1.         1.        ]
 [0.5        1.         1.         1.        ]
 [0.5        1.         1.         1.        ]]

comparison
_______________________________________________

#synthesis

In [130]:
copy_df = original_df.copy(deep=True)
summation = copy_df.sum()
print(summation)

price       2.333333
mpg        12.000000
comfort     5.250000
style       3.750000
dtype: float64


In [131]:
copy_df

,price,mpg,comfort,style
price,1.000000,3.0,2.00,2.00
mpg,0.333333,1.0,0.25,0.25
comfort,0.500000,4.0,1.00,0.50
style,0.500000,4.0,2.00,1.00


In [132]:
priority_vector = []
cols = list(original_df.columns)
for row_index in range(len(copy_df)):
  for i in range(len(cols)):
    copy_df.iloc[row_index][i] = copy_df.iloc[row_index, i] / summation[cols[i]]
  priority_vector.append(copy_df.iloc[row_index].sum()/len(cols))
print(priority_vector)


[0.39821428571428574, 0.08511904761904762, 0.21785714285714286, 0.2988095238095238]


In [133]:
copy_df['priority_vector'] = priority_vector

In [134]:
copy_df

,price,mpg,comfort,style,priority_vector
price,0.428571,0.250000,0.380952,0.533333,0.398214
mpg,0.142857,0.083333,0.047619,0.066667,0.085119
comfort,0.214286,0.333333,0.190476,0.133333,0.217857
style,0.214286,0.333333,0.380952,0.266667,0.298810


#consinstency ratio

In [135]:
consistency = np.dot(original_df.to_numpy(), np.array(priority_vector))
print(consistency)

[1.68690476 0.34702381 0.90684524 1.27410714]


### divide it with priority vector

In [136]:
lambda_max = (consistency / np.array(priority_vector)).mean()
print(lambda_max)

4.18490224979141


consistency index

In [137]:
C_I = (lambda_max - len(analytica.criteria)) / (len(analytica.criteria) - 1)
print(C_I)

0.0616340832638033


In [138]:
all_rgb = [0.58, 0.90, 1.12, 1.24, 1.32, 1.41]
r_i = {i:[i] for i in range(3, 9)}
C_ratio = C_I / all_rgb[len(analytica.choices)]

In [139]:
C_ratio

0.049704905857905886

# Choices pairwise matrix

In [141]:
choice_pairwise2 = []
choices_pairwise = AHP(choices=['Accord', 'Saturn', 'Cavalier'],
                       criteria=['Accord', 'Saturn', 'Cavalier'])

for criteria in analytica.criteria:
    print(f"CRITERIA: {criteria.upper()}")
    print("*" * 50)
    arr, original_df2 = choices_pairwise.Pairwise1()
    choice_pairwise2.append(original_df2.copy())  # Make sure to append a copy of arr
    # print("FAKE: ", arr)

# Now choice_pairwise2 should contain distinct arrays.


CRITERIA: PRICE
**************************************************
comparison
____________________________________________________________

CRITERIA
0. Accord
1. Saturn
2. Cavalier

Accord - Saturn :1
rating out of 9 : 3


CRITERIA
0. Accord
1. Saturn
2. Cavalier

Accord - Cavalier :2
rating out of 9 : 4

comparison
____________________________________________________________

CRITERIA
0. Accord
1. Saturn
2. Cavalier

Saturn - Cavalier :2
rating out of 9 : 2

comparison
____________________________________________________________
CRITERIA: MPG
**************************************************
comparison
____________________________________________________________

CRITERIA
0. Accord
1. Saturn
2. Cavalier

Accord - Saturn :1
rating out of 9 : 4


CRITERIA
0. Accord
1. Saturn
2. Cavalier

Accord - Cavalier :2
rating out of 9 : 6

comparison
____________________________________________________________

CRITERIA
0. Accord
1. Saturn
2. Cavalier

Saturn - Cavalier :2
rating out of 9 : 3

co

In [142]:
choice_pairwise2

[          Accord    Saturn  Cavalier
 Accord       1.0  0.333333      0.25
 Saturn       3.0  1.000000      0.50
 Cavalier     4.0  2.000000      1.00,
           Accord  Saturn  Cavalier
 Accord       1.0    0.25  0.166667
 Saturn       4.0    1.00  0.333333
 Cavalier     6.0    3.00  1.000000,
             Accord  Saturn  Cavalier
 Accord    1.000000     6.0  8.000000
 Saturn    0.166667     1.0  0.333333
 Cavalier  0.125000     3.0  1.000000,
           Accord    Saturn  Cavalier
 Accord      1.00  0.333333       4.0
 Saturn      3.00  1.000000       7.0
 Cavalier    0.25  0.142857       1.0]

# choice synthesis

In [ ]:
pair_copy = cp(choice_pairwise2)
print(f"ALL DF: {pair_copy}")
overall_priorities = []
for choiceDf in pair_copy:
  choice_df = choiceDf.copy(deep=True)
  summation = choice_df.sum()
  # print(summation)

  priority_v = []
  cols = list(choiceDf.columns)
  for row_index in range(len(choice_df)):
    for i in range(len(cols)):
      choice_df.iloc[row_index][i] = choice_df.iloc[row_index, i] / summation[cols[i]]
    priority_v.append(choice_df.iloc[row_index].sum()/len(cols))
  print(f"prio: {priority_v}")
  print(f"df: {choiceDf.head()}")
  # APPEND TO OVERALL

  overall_priorities.append(priority_v)

  # print(overall_prio_df)

In [144]:
overall_prio_df = pd.DataFrame(np.array(overall_priorities).transpose())
overall_prio_df

,0,1,2,3
0,0.122619,0.086948,0.743779,0.264811
1,0.320238,0.273718,0.088249,0.655545
2,0.557143,0.639335,0.167972,0.079644


In [145]:
composite_priority = np.dot(overall_prio_df.to_numpy(),
                            np.array(priority_vector).transpose())
composite_priority

array([0.29739524, 0.3659307 , 0.33667406])